In [38]:
import time
notebook_start_time = time.perf_counter()

# PRIDE Negative peptide data processing

## Imports

### Built-in imports

In [39]:
import time
import pickle
from pathlib import Path
import gzip
import re
import bisect
import uuid

### Shared library imports

In [40]:
from glyc_processing import cf
from glyc_processing.data_formats.common.config import BaseConfig

from glyc_processing.misc import download, display_invalid_rows

from glyc_processing.uniprot import get_uniprot_id_mappings, get_uniprot_entries, get_uniprot_isoforms, \
    get_entry_isoforms_dicts, set_new_uniprot_ids, set_missing_peptide_sequences, set_uniprot_isoforms_containing_peptides, \
    set_missing_peptide_sequences, check_uniprot_idmapping, consistent_entry_peptide, consistent_entry_sites, \
    set_correct_peptide_ranges, split_uniprot_id, consistent_entry_peptide, get_uniprot_id_mappings, check_uniprot_idmapping, \
    set_new_uniprot_ids

from glyc_processing.data_formats.common.validation import valid_uniprot, valid_peptide_id, valid_peptide, valid_peptide_range, \
    consistent_peptide_length

from glyc_processing.clustering import cluster_peptides

### External imports

In [41]:
import requests
import untangle
import numpy as np
import pandas as pd
from matplotlib import pyplot
from tqdm.auto import tqdm
from IPython.display import display

## Paths & Constants

In [42]:
#BASE_DIR = Path("/home/jakob/Masters/Data")
BASE_DIR = Path("/home/jakob/Cloudvault_new/Data")

# List of used uniprot IDs in positive data
CLEAN_POSITIVE_UNIPROT_IDS_FILE = BASE_DIR/'NetOGlyc5 data'/'GalNAc data'/'02-GalNAC_processing'/'uniprot_ids.txt'

# All peptides found for the organisms reference proteome (will be downloaded if not present)
# As querying PRIDE is slow, it is best to first download all peptides we might need
ALL_PEPTIDES_FILE = BASE_DIR/'NetOGlyc5 data'/'GalNAc data'/'03-PRIDE_processing'/'all_human_peptides.pkl.gz'

# Minimum needed PSMs to include peptide in data
PEPTIDE_MINIMUM_PSMS = 100

# Minimum needed projects to include peptide in data
PEPTIDE_MINIMUM_PROJECTS = 3

# Path to write cleaned data file to
CLEAN_NEGATIVE_DATA_FILE = BASE_DIR/'NetOGlyc5 data'/'GalNAc data'/'03-PRIDE_processing'/'clean_data.xlsx'

# Path to write list of used uniprot IDs to
CLEAN_NEGATIVE_UNIPROT_IDS_FILE = BASE_DIR/'NetOGlyc5 data'/'GalNAc data'/'03-PRIDE_processing'/'uniprot_ids.txt'

In [43]:
# Set right config
cf.use_config(BaseConfig)

In [44]:
# The temp folder is used to organize data-specific temporary files
cf.TEMP_DIR = BASE_DIR/'uniprot'/'PRIDE'

# Uniprot release downloads directory path (make sure you have a few GB of free space)
cf.UNIPROT_DOWNLOADS_DIR = BASE_DIR/'uniprot'

# The amino acids that are allowed to be glycosylated
cf.ALLOWED_AA = ('S','T','Y')

# The uniport release can be 'latest' for the current release or any of those with format (YYYY_MM) found here: https://ftp.uniprot.org/pub/databases/uniprot/previous_releases/
# Warning: Uniprot only keeps previous releases other than the first of the year for 2 years, so only use first yearly (2015_01, 2021_01 etc.) for reproducability!
cf.UNIPROT_RELEASE = '2021_01'

# If True ignores existing data-specific temp files and recreates them from scratch
# This should be used if the data or script has changed
cf.IGNORE_EXISTING_FILES = True

In [45]:
print(f"Using Uniprot Release {cf.TRUE_UNIPROT_RELEASE}")

Using Uniprot Release 2021_01


## Download proteome IDs from Uniprot & compare with data

In [46]:
# Get uniprot IDs of full human proteome

params = {
    "query": "reviewed:yes AND proteome:up000005640", # We begrudgingly use the proteome list from the current Uniprot release, as the file for previous releases is 128GB!
    "format": "list",
}
all_proteins_string = download(url='https://www.uniprot.org/uniprot', query_params=params)
all_proteins = set(all_proteins_string.strip().split('\n'))
print(f"Number of proteins in human proteome: {len(all_proteins)}")

Number of proteins in human proteome: 20360


In [47]:
# Get uniprot IDs of all O-glycosylatable human proteins

subcellular_locations = (
    "Endoplasmic reticulum [SL-0095]",
    "COPI-coated vesicle [SL-0075]",
    "COPII-coated vesicle [SL-0077]",
    "Endoplasmic reticulum-Golgi intermediate compartment [SL-0098]",
    "Golgi apparatus [SL-0132]",
    "Secretory vesicle [SL-0244]",
    "Clathrin-coated vesicle [SL-0070]",
    "Endosome [SL-0101]",
    "Cell membrane [SL-0039]",
    "Cell surface [SL-0310]",
    "Coated pit [SL-0072]",
    "Cell junction [SL-0038]",
    "Secreted [SL-0243]",
    "Microvillus [SL-0293]",
    "Cell projection [SL-0280]",
)

locations_string = ' OR '.join(f'locations:(location:"{location}")' for location in subcellular_locations)

params = {
    "query": f'reviewed:yes AND proteome:up000005640 AND ({locations_string})',
    "format": "list",
}

glycosylatable_proteins_string = download(url='https://www.uniprot.org/uniprot', query_params=params)
glycosylatable_proteins = set(glycosylatable_proteins_string.strip().split('\n'))
print(f"Number of human proteins in the secretory (O-glycosylation) pathway in Uniprot: {len(glycosylatable_proteins)}")

Number of human proteins in the secretory (O-glycosylation) pathway in Uniprot: 7932


In [11]:
# Get uniprot IDs of proteins identified in cleaned data

with open(CLEAN_POSITIVE_UNIPROT_IDS_FILE, 'r') as f:
    positive_proteins = {split_uniprot_id(full_id)[0] for full_id in f.read().split('\n')}
print(f"Number of proteins in cleaned data (positive_proteins): {len(positive_proteins)}")

Number of proteins in cleaned data (positive_proteins): 4684


In [12]:
# Get uniprot IDs of O-glycosylatable proteins and those seen in cleaned data

wanted_proteins = glycosylatable_proteins | positive_proteins

print(f"Number of proteins only found in glycosylatable_proteins: {len(glycosylatable_proteins - positive_proteins)}")
print(f"Number of proteins only found in positive_proteins: {len(positive_proteins - glycosylatable_proteins)}")
print(f"Number of proteins found in both glycosylatable_proteins & positive_proteins: {len(glycosylatable_proteins & positive_proteins)}")
print(f"Number of proteins found in either glycosylatable_proteins & positive_proteins: {len(wanted_proteins)}")

Number of proteins only found in glycosylatable_proteins: 5519
Number of proteins only found in positive_proteins: 2394
Number of proteins found in both glycosylatable_proteins & positive_proteins: 2290
Number of proteins found in either glycosylatable_proteins & positive_proteins: 10203


## Download UNIMOD DB & Extract O-glyc IDs

In [13]:
# Download UNIMOD database and extract all O-linked glycosylation modification IDs and their possible AA sites

unimod = untangle.parse('https://www.unimod.org/xml/unimod.xml')
olinked_mods = {}
for umod_mod in unimod.umod_unimod.umod_modifications.umod_mod:
    record_id = int(umod_mod['record_id'])
    
    for umod_specificity in umod_mod.umod_specificity:
        if umod_specificity['classification'] == "O-linked glycosylation":
            if umod_specificity['site'] not in cf.ALLOWED_AA:
                print(f"UNIMOD:{record_id} - Site '{umod_specificity['site']}' is not part of allowed amino acids")
            if umod_specificity['position'] != 'Anywhere':
                print(f"UNIMOD:{record_id} - Position '{umod_specificity['position']}' is not the expected 'Anywhere'")
            if record_id not in olinked_mods:
                olinked_mods[record_id] = [umod_specificity['site']]
            else:
                olinked_mods[record_id].append(umod_specificity['site'])
olinked_mods

{41: ['T', 'S', 'Y'],
 43: ['T', 'S'],
 54: ['T', 'S'],
 142: ['T', 'S'],
 143: ['T', 'S'],
 144: ['T', 'S'],
 146: ['T', 'S'],
 148: ['T', 'S'],
 149: ['T', 'S'],
 152: ['T', 'S'],
 153: ['T', 'S'],
 156: ['T', 'S'],
 158: ['T', 'S'],
 159: ['T', 'S'],
 160: ['T', 'S'],
 213: ['T', 'S'],
 2022: ['T', 'S'],
 295: ['T', 'S'],
 305: ['T', 'S'],
 307: ['T', 'S'],
 309: ['T', 'S'],
 310: ['T', 'S'],
 311: ['T', 'S'],
 428: ['T', 'S'],
 429: ['T', 'S'],
 454: ['T', 'S'],
 490: ['T', 'S'],
 512: ['S', 'T'],
 793: ['T', 'S'],
 1303: ['T', 'S'],
 1304: ['T', 'S'],
 1367: ['T', 'S'],
 1375: ['T', 'S'],
 1376: ['T', 'S'],
 1377: ['T', 'S'],
 1378: ['T', 'S'],
 1379: ['T', 'S'],
 1412: ['T', 'S'],
 1413: ['T', 'S'],
 1428: ['T', 'S'],
 1425: ['T', 'S'],
 1426: ['T', 'S'],
 1427: ['T', 'S'],
 1429: ['T', 'S'],
 1430: ['T', 'S'],
 1431: ['T', 'S'],
 1432: ['T', 'S'],
 1433: ['T', 'S'],
 1434: ['T', 'S'],
 1435: ['T', 'S'],
 1444: ['T', 'S'],
 1436: ['T', 'S'],
 1437: ['T', 'S'],
 1438: ['T', 'S'],


## Download Peptides from PRIDE

In [14]:
# (Down)load all human peptides from PRIDE /peptidesummary endpoint (per uniprot ID)

if ALL_PEPTIDES_FILE.exists():
    with gzip.open(ALL_PEPTIDES_FILE, 'rb') as f:
        print("Loading human PRIDE peptides")
        all_peptides = pickle.load(f)
else:
    all_peptides = []
    for protein in tqdm(iterable=all_proteins, desc="Downloading human PRIDE peptides"):
        page = 0
        while True:
            peptide_summary = requests.get(f'https://www.ebi.ac.uk/pride/ws/archive/v2/peptidesummary/peptide?proteinAccession={protein}&page={page}').json()
            if '_embedded' not in peptide_summary or page+1 >= peptide_summary['page']['totalPages']:
                break
            all_peptides.extend(peptide_summary['_embedded']['peptideSummaries'])
            page += 1
    with gzip.open(ALL_PEPTIDES_FILE, 'wb') as f:
        pickle.dump(all_peptides, f, protocol=4)

wanted_peptides = [peptide for peptide in all_peptides if peptide['proteinAccession'] in wanted_proteins]
print(f"Number of peptides from wanted proteins: {len(wanted_peptides)}")

Loading human PRIDE peptides
Number of peptides from wanted proteins: 293000


In [15]:
# Filter out peptides that are not unique to humans
# Split peptides into non-O-glycosylated, O-glycosylated and unusual O-glycosylated (seemingly errors?)

olinked_peptides = []
unusual_olinked_peptides = []
non_olinked_peptides = []

# PRIDE projects seen for each peptide type
olinked_projects = set()
unusual_projects = set()
non_olinked_projects = set()

for i, peptide in enumerate(wanted_peptides):
    seq = peptide['peptideSequence']
    has_oglyc_ptm = False
    has_unusual_oglyc_ptm = False
    for ptm in peptide['ptmsMap']:
        unusual_oglyc_ptm = False
        unimod_id, *unimod_name, unimod_pos = ptm.split(',')
        unimod_id = int(unimod_id.split(':')[1])
        unimod_name = ','.join(unimod_name)
        unimod_pos = int(unimod_pos)

        if unimod_id in olinked_mods:
            if unimod_pos < 1 or unimod_pos > len(seq):
                unusual_oglyc_ptm = True
                print(f"terminal O-glyc - index: {i}, ptm: {ptm}, projects: {peptide['ptmsMap'][ptm]}, seq: {seq}")
            elif seq[unimod_pos-1] not in olinked_mods[unimod_id]:
                unusual_oglyc_ptm = True
                print(f"O-glyc AA not found for UNIMOD ID - index: {i}, ptm: {ptm}, projects: {peptide['ptmsMap'][ptm]}, seq: {seq}, aa: {seq[unimod_pos-1]}")
            elif seq[unimod_pos-1] not in cf.ALLOWED_AA:
                unusual_oglyc_ptm = True
                print(f"non-allowed AA O-glyc - index: {i}, ptm: {ptm}, projects: {peptide['ptmsMap'][ptm]}, seq: {seq}, aa: {seq[unimod_pos-1]}")
            
            if unusual_oglyc_ptm:
                has_unusual_oglyc_ptm = True
                unusual_projects.update(peptide['ptmsMap'][ptm])
            else:
                has_oglyc_ptm = True
                olinked_projects.update(peptide['ptmsMap'][ptm])
            
    
    if has_unusual_oglyc_ptm:
        unusual_olinked_peptides.append(peptide)
    elif has_oglyc_ptm:
        olinked_peptides.append(peptide)
    else:
        non_olinked_peptides.append(peptide)

print(f"olinked_peptides={len(olinked_peptides)}, unusual_olinked_peptides={len(unusual_olinked_peptides)}, non_olinked_peptides={len(non_olinked_peptides)}")

O-glyc AA not found for UNIMOD ID - index: 9, ptm: UNIMOD:43,HexNAc,6, projects: ['PXD007700'], seq: SQGSGNEAEPLGK, aa: N
O-glyc AA not found for UNIMOD ID - index: 2378, ptm: UNIMOD:43,HexNAc,4, projects: ['PXD007700'], seq: QLINALQINNTAVGHALVLPAGR, aa: N
O-glyc AA not found for UNIMOD ID - index: 2407, ptm: UNIMOD:43,HexNAc,6, projects: ['PXD007700'], seq: GPPGVNGTQGFQGCPGQR, aa: N
O-glyc AA not found for UNIMOD ID - index: 2453, ptm: UNIMOD:43,HexNAc,14, projects: ['PXD007700'], seq: VAVVQHAPSESVDNASMPPVK, aa: N
O-glyc AA not found for UNIMOD ID - index: 2471, ptm: UNIMOD:43,HexNAc,3, projects: ['PXD007700'], seq: IMNSFGPSAATPAPPGVDTPPPSRPEK, aa: N
O-glyc AA not found for UNIMOD ID - index: 2536, ptm: UNIMOD:43,HexNAc,2, projects: ['PXD007700'], seq: QNLTVTDR, aa: N
O-glyc AA not found for UNIMOD ID - index: 2566, ptm: UNIMOD:43,HexNAc,3, projects: ['PXD007700'], seq: IMNSFGPSAATPAPPGVDTPPPSRPEKK, aa: N
O-glyc AA not found for UNIMOD ID - index: 2616, ptm: UNIMOD:43,HexNAc,18, proje

### Convert peptides to Dataframe

In [16]:
def get_peptide_data():
    for peptide in non_olinked_peptides:
        # We filter out peptides that are:
        #     seen in other organisms, seen in too few projects,
        #     has too few PSMs or no glycosylatable amino acids
        if (
            peptide['isMultiOrganism']
            or len(peptide['projectAccessions']) < PEPTIDE_MINIMUM_PROJECTS
            or peptide['psmsCount'] < PEPTIDE_MINIMUM_PSMS
            or not any(aa in peptide['peptideSequence'] for aa in cf.ALLOWED_AA)
        ):
            continue
        
        yield peptide['proteinAccession'], str(uuid.uuid4()), peptide['peptideSequence'], np.nan, np.nan, peptide['psmsCount'], len(peptide['projectAccessions'])

df = pd.DataFrame(get_peptide_data(), columns=['uniprot', 'peptide_id', 'peptide', 'peptide_start', 'peptide_end', 'psmsCount', 'projectAccessionsCount'])
n_original_rows = len(df)

In [17]:
display(df)
display(df.info())

,uniprot,peptide_id,peptide,peptide_start,peptide_end,psmsCount,projectAccessionsCount
0,Q9NUQ6,b68b7cc9-9031-4695-a1b3-18967f2bf5d6,MVSSLPSTADPSHQTMPANK,NaN,NaN,501,27
1,Q9NUQ6,219d81ec-7a9c-4b6b-b071-8abe9261c22b,DSSSTDSANEKPALIPR,NaN,NaN,441,25
2,Q9NUQ6,d6f45eba-c073-47b3-836f-31edb80e23d2,TPEAPAHSEKPR,NaN,NaN,284,42
3,Q9NUQ6,c32ed75f-c291-4881-96fe-f15cb3b75039,SDGLQWSAEQPCNPSKPK,NaN,NaN,271,14
4,Q9NUQ6,40a287aa-6ae3-404c-a1e6-7680087a41b1,AVQAFVDGSAIQVLK,NaN,NaN,246,23
...,...,...,...,...,...,...,...
12799,P17661,1696e9b3-d875-4058-af15-b1a880cd7a9f,RTFGGAPGFPLGSPLSSPVFPR,NaN,NaN,236,29
12800,P17661,da0d6e71-85aa-4b37-b4fa-9173a2cc3902,FASEASGYQDNIARLEEEIR,NaN,NaN,114,13
12801,P17661,9165343a-0688-4c44-a407-4ceaa30d4c5d,LLEGEESRINLPIQTYSALNFR,NaN,NaN,105,4
12802,P17661,6ea2fa64-913b-43a0-b149-deb5985a67d0,INLPIQTYSALNFRETSPEQR,NaN,NaN,104,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12804 entries, 0 to 12803
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uniprot                 12804 non-null  object 
 1   peptide_id              12804 non-null  object 
 2   peptide                 12804 non-null  object 
 3   peptide_start           0 non-null      float64
 4   peptide_end             0 non-null      float64
 5   psmsCount               12804 non-null  int64  
 6   projectAccessionsCount  12804 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 700.3+ KB


None

## Mapping to UniProt sequences

### Fetch any new UniProt ID Mappings

In [18]:
mappings_dict = get_uniprot_id_mappings(set(df['uniprot'].unique()))

Extracting ID mappings from /home/jakob/Cloudvault_new/Data/uniprot/uniprot_sprot2021_01.dat.bgz:   0%|       …

### Check if any IDs didn't map to uniprot or have been merged/demerged/deleted and change them accordingly

In [19]:
check_uniprot_idmapping(set(df['uniprot'].unique()), mappings_dict)

{'new_mapping_ids': set(), 'non_mapping_ids': set()}

In [20]:
set_new_uniprot_ids(df, mappings_dict)

In [21]:
valid_uniprot_rows = display_invalid_rows(df, "Deleted 'uniprot' IDs", valid_uniprot)
df = df[valid_uniprot_rows].reset_index(drop=True)

### Fetch UniProt sequences

In [22]:
get_uniprot_entries(set(df['uniprot'].unique()))
get_uniprot_isoforms(set(df['uniprot'].unique()))
entry_isoforms, isoform_seqs = get_entry_isoforms_dicts()
print(f"Uniprot sequences - Number of entries: {len(entry_isoforms)}, Number of isoforms: {len(isoform_seqs)}")

Extracting data-specific Uniprot entries from /home/jakob/Cloudvault_new/Data/uniprot/uniprot_sprot2021_01.dat…

Extracting data-specific Uniprot entry isoforms from /home/jakob/Cloudvault_new/Data/uniprot/uniprot_sprot_var…

Uniprot sequences - Number of entries: 1225, Number of isoforms: 3497


### Recover missing peptide ranges

In [23]:
missing_peptide_range_count = (df['peptide'].notna() & df['peptide_start'].isna() & df['peptide_end'].isna()).sum()
print(f"Missing peptide ranges before: {missing_peptide_range_count} {missing_peptide_range_count/df.shape[0]*100:.2f}%")

set_correct_peptide_ranges(df, isoform_seqs)

missing_peptide_range_count = (df['peptide'].notna() & df['peptide_start'].isna() & df['peptide_end'].isna()).sum()
print(f"Missing peptide ranges after: {missing_peptide_range_count} {missing_peptide_range_count/df.shape[0]*100:.2f}%")

Missing peptide ranges before: 12804 100.00%
Missing peptide ranges after: 122 0.95%


### Set the right isoform (& peptide/site position info) for each peptide

In [24]:
print(f"Non-canonical isoform sites before: {df['uniprot'].str.contains('-').sum()}, Unique: {df.loc[df['uniprot'].str.contains('-'), 'uniprot'].nunique()}")

set_uniprot_isoforms_containing_peptides(df, entry_isoforms, isoform_seqs)

print(f"Non-canonical isoform sites after: {df['uniprot'].str.contains('-').sum()}, Unique: {df.loc[df['uniprot'].str.contains('-'), 'uniprot'].nunique()}")

Non-canonical isoform sites before: 0, Unique: 0
Non-canonical isoform sites after: 0, Unique: 0


## Data Validation

### Basic column validation

In [25]:
def validate_individual_columns(df):
    valid_rows = display_invalid_rows(df, "Invalid 'uniprot' rows", valid_uniprot)
    valid_rows &= display_invalid_rows(df, "Invalid 'peptide_id' rows", valid_peptide_id)
    valid_rows &= display_invalid_rows(df, "Invalid 'peptide' rows", valid_peptide, na_allowed=False)
    valid_rows &= display_invalid_rows(df, "Invalid 'peptide_start' and/or 'peptide_end' rows", valid_peptide_range, na_allowed=False)
    return valid_rows

In [26]:
valid_rows = validate_individual_columns(df)

#### Invalid 'peptide_start' and/or 'peptide_end' rows: 122 (0.95%):

,uniprot,peptide_id,peptide,peptide_start,peptide_end,psmsCount,projectAccessionsCount
241,Q9Y6R7,6aa8b524-78f3-466e-9fa7-78195167ee3d,LEQYEGPGFCGPLAPGTGGPFTTCHAHVPPESFFK,NaN,NaN,131,3
838,Q96Q06,25ea13bc-fb9e-4018-a8dc-c90b15b17bda,DTVCSGVTGAANVAK,NaN,NaN,163,10
869,Q9HC84,a54cb7a7-d346-4d90-b225-d6e70918062c,MCFNYEIR,NaN,NaN,319,26
873,Q9HC84,65535f68-8bdc-44c2-9d0d-40e898727672,ELGQVVECSLDFGLVCR,NaN,NaN,262,20
900,Q9HC84,1c108775-b461-4646-a6b6-2adad4ce9a2c,FKMCFNYEIR,NaN,NaN,117,3
...,...,...,...,...,...,...,...
9737,P12036,3d087b0b-8265-4d3f-8816-9777be540ee3,SPVKEEAKSPEK,NaN,NaN,136,4
10192,P02751,ce57e14b-74c8-4dc0-a5d7-15b1fe116e09,ESVPISDTIIPAVPPPTDLR,NaN,NaN,1556,61
10235,P02751,a0c5decc-7226-4cbf-bba9-cf9fd816316b,ESVPISDTIIPAVPPPTDLRFTNIGPDTMR,NaN,NaN,107,4
11460,P00738,8c878e4a-00ec-4e4d-a7bb-f2826fb2a94f,YQCKNYYK,NaN,NaN,168,12


### Field consistency validation

In [27]:
def check_field_consistency(df):
    consistent_rows = display_invalid_rows(df, "Inconsistent 'peptide' length compared to 'peptide_start' & 'peptide.end' rows", consistent_peptide_length, na_allowed=False)
    return consistent_rows

In [28]:
consistent_rows = check_field_consistency(df)

### UniProt entries data consistency validation

In [29]:
def validate_to_entries(df, isoform_seqs):
    entry_valid_rows = display_invalid_rows(df, "Non-matching peptide sequence between data and UniProt sequence", consistent_entry_peptide, isoform_seqs = isoform_seqs)
    return entry_valid_rows

In [30]:
entry_valid_rows = validate_to_entries(df, isoform_seqs)

## Create clean dataframe

In [31]:
valid_and_consistent_rows = valid_rows & consistent_rows & entry_valid_rows

clean_df = df[valid_and_consistent_rows].reset_index(drop=True).copy()

#clean_df['peptide_start'] = clean_df['peptide_start'].astype(pd.Int32Dtype())
#clean_df['peptide_end'] = clean_df['peptide_end'].astype(pd.Int32Dtype())

print(f"Clean dataframe has {len(clean_df)} rows, {(n_original_rows-len(clean_df))/n_original_rows*100:4.2f}% of original rows were discarded")

Clean dataframe has 12682 rows, 0.95% of original rows were discarded


In [32]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12682 entries, 0 to 12681
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uniprot                 12682 non-null  object 
 1   peptide_id              12682 non-null  object 
 2   peptide                 12682 non-null  object 
 3   peptide_start           12682 non-null  float64
 4   peptide_end             12682 non-null  float64
 5   psmsCount               12682 non-null  int64  
 6   projectAccessionsCount  12682 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 693.7+ KB


### Check how redundant peptides are

In [33]:
representative_rows, redundant_rows = cluster_peptides(clean_df)
print(f"Cluster representatives: {sum(representative_rows)} {sum(representative_rows)/clean_df.shape[0]*100}%")
print(f"Redundant rows: {sum(redundant_rows)} {sum(redundant_rows)/clean_df.shape[0]*100}%")
print(f"Invalid peptide rows: {clean_df.shape[0]-sum(representative_rows | redundant_rows)} {(clean_df.shape[0]-sum(representative_rows | redundant_rows))/clean_df.shape[0]*100}%")

Cluster representatives: 3995 31.501340482573724%
Redundant rows: 8687 68.49865951742628%
Invalid peptide rows: 0 0.0%


### Save results

In [34]:
clean_df.to_excel(CLEAN_NEGATIVE_DATA_FILE, index=False)

uniprot_ids_string = '\n'.join(sorted(clean_df['uniprot'].unique()))
with open(CLEAN_NEGATIVE_UNIPROT_IDS_FILE, 'w') as f:
    f.write(uniprot_ids_string)

In [37]:
clean_negative_proteins = set(clean_df['uniprot'].unique())

print(f"Number of proteins only found in clean_negative_proteins: {len(clean_negative_proteins - positive_proteins)}")
print(f"Number of proteins only found in positive_proteins: {len(positive_proteins - clean_negative_proteins)}")
print(f"Number of proteins found in both clean_negative_proteins & positive_proteins: {len(clean_negative_proteins & positive_proteins)}")
print(f"Number of proteins found in either clean_negative_proteins & positive_proteins: {len(clean_negative_proteins | positive_proteins)}")

Number of proteins only found in clean_negative_proteins: 311
Number of proteins only found in positive_proteins: 3770
Number of proteins found in both clean_negative_proteins & positive_proteins: 914
Number of proteins found in either clean_negative_proteins & positive_proteins: 4995


In [35]:
notebook_end_time = time.perf_counter()
print(f"Notebook took {notebook_end_time-notebook_start_time} seconds to run")

Notebook took 277.97103307299994 seconds to run
